In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import numpy as np
data = pd.read_csv(r"..\Data\latest_ADS 1.csv")
data = data.drop(['Unnamed: 0', 'finalspend_usd', "effectiveness_usd", "starttime"], axis = 1)

# Load the model
from catboost import CatBoostRegressor
loaded_model = CatBoostRegressor()
zone="APAC"
loaded_model.load_model(f"model_Arun_zone_{zone}.cbm")


In [23]:
data = data[loaded_model.feature_names_ + ["data","event_id", "efficiency"]]

In [24]:
train = data[data["data"]== 'train']
forecast = data[data['data'] == 'submission_df']

In [25]:
forecast

,reviewtimeperiod,biddingperiod,overtimeperiod(minutes),bidrankthattriggersovertime,canparticipantsplacebidsduringpreviewperiod_Do not allow prebids,improvebidamountby_Percentage,"showlineitemlevelrankinlot_Yes, to Buyers and Participants",enabletrafficlightbidding_Yes,setareviewperiodafterlotoritemcloses_Yes,setareviewperiodafterlotcloses_Yes,...,Count_supplier,participant,bestbid_usd,Mean_Lot_Item_Bid_Version,itemno,baselinespend_usd,item_name,data,event_id,efficiency
15011,2.0,20.0,5.0,3.0,0,1,1,0,1,0,...,440097,3.0,40823.71,108.800247,1.0,35011.76,1,submission_df,AXJW,0.0
15012,2.0,30.0,5.0,4.0,0,1,1,0,1,0,...,440097,4.0,25644.36,108.800247,1.0,26736.25,1,submission_df,AQTQ,0.0
15013,2.0,30.0,5.0,4.0,0,1,0,1,1,0,...,440097,4.0,16253.81,108.800247,1.0,14195.47,1,submission_df,DTOB,0.0
15014,2.0,30.0,2.0,1.0,0,0,0,1,1,0,...,311963,7.0,801000.00,236.873956,1.0,750000.00,1,submission_df,NVF,0.0
15015,2.0,15.0,5.0,3.0,0,0,1,0,1,0,...,440097,3.0,12962.38,108.800247,1.0,12000.00,1,submission_df,AXHF,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19708,2.0,0.0,1.0,1.0,1,0,0,1,1,0,...,187451,22.0,295250.50,135.246032,2.0,500489.70,2,submission_df,AIAA,0.0
19709,2.0,45.0,5.0,2.0,0,1,0,1,1,0,...,349812,4.0,36523.77,77.891784,2.0,36523.77,2,submission_df,DYSS,0.0
19710,2.0,30.0,5.0,2.0,1,0,0,0,0,1,...,349812,12.0,3442025.08,77.891784,3.0,3442025.08,3,submission_df,UGX,0.0
19711,2.0,30.0,2.0,1.0,1,1,1,0,0,1,...,349812,5.0,116631.46,77.891784,1.0,116631.46,1,submission_df,EEEM,0.0


In [26]:
binary_variables = [
    'canparticipantsplacebidsduringpreviewperiod_Do not allow prebids',
    'improvebidamountby_Percentage',
    'showlineitemlevelrankinlot_Yes, to Buyers and Participants',
    'enabletrafficlightbidding_Yes',
    'setareviewperiodafterlotoritemcloses_Yes',
    'setareviewperiodafterlotcloses_Yes',
    'canparticipantsseeranks?_Their own rank',
    'canparticipantsseeranks?_No',
]

continuous_variables = [
    'reviewtimeperiod',
    'biddingperiod', 
    'overtimeperiod(minutes)',
    'bidrankthattriggersovertime',
]

other_variables = [
    'baselinespend_usd',
    # 'zone_APAC',
    'Count_supplier',
    'event_template_ABI Reverse Auction - NI',
    'categ_l2_IND CAPEX',
    'item_name',
    'Mean_Lot_Item_Bid_Version',
    'ratio',
    'itemno',
    'bestbid_usd',
    'participant',
    'Count_Item_Name'
]
X_forecast = forecast.drop(['data', 'event_id'], axis = 1)
X_forecast

,reviewtimeperiod,biddingperiod,overtimeperiod(minutes),bidrankthattriggersovertime,canparticipantsplacebidsduringpreviewperiod_Do not allow prebids,improvebidamountby_Percentage,"showlineitemlevelrankinlot_Yes, to Buyers and Participants",enabletrafficlightbidding_Yes,setareviewperiodafterlotoritemcloses_Yes,setareviewperiodafterlotcloses_Yes,...,Count_Item_Name,ratio,Count_supplier,participant,bestbid_usd,Mean_Lot_Item_Bid_Version,itemno,baselinespend_usd,item_name,efficiency
15011,2.0,20.0,5.0,3.0,0,1,1,0,1,0,...,440097,3.0,440097,3.0,40823.71,108.800247,1.0,35011.76,1,0.0
15012,2.0,30.0,5.0,4.0,0,1,1,0,1,0,...,440097,4.0,440097,4.0,25644.36,108.800247,1.0,26736.25,1,0.0
15013,2.0,30.0,5.0,4.0,0,1,0,1,1,0,...,440097,4.0,440097,4.0,16253.81,108.800247,1.0,14195.47,1,0.0
15014,2.0,30.0,2.0,1.0,0,0,0,1,1,0,...,311963,7.0,311963,7.0,801000.00,236.873956,1.0,750000.00,1,0.0
15015,2.0,15.0,5.0,3.0,0,0,1,0,1,0,...,440097,3.0,440097,3.0,12962.38,108.800247,1.0,12000.00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19708,2.0,0.0,1.0,1.0,1,0,0,1,1,0,...,187451,11.0,187451,22.0,295250.50,135.246032,2.0,500489.70,2,0.0
19709,2.0,45.0,5.0,2.0,0,1,0,1,1,0,...,349812,2.0,349812,4.0,36523.77,77.891784,2.0,36523.77,2,0.0
19710,2.0,30.0,5.0,2.0,1,0,0,0,0,1,...,349812,4.0,349812,12.0,3442025.08,77.891784,3.0,3442025.08,3,0.0
19711,2.0,30.0,2.0,1.0,1,1,1,0,0,1,...,349812,5.0,349812,5.0,116631.46,77.891784,1.0,116631.46,1,0.0


In [27]:
from scipy.optimize import minimize
import numpy as np

def objective_function_scipy(vars, other_values, loaded_model, continuous_count, binary_count):
    """
    Objective function for optimization.
    """
    continuous_vars = vars[:continuous_count]
    binary_vars = vars[continuous_count:]
    # binary_vars = np.round(vars[continuous_count:])  # Ensure binary variables are rounded to 0 or 1
    candidate = list(continuous_vars) + list(binary_vars) + other_values
    # print(candidate)
    # print(loaded_model.predict(candidate))
    return -loaded_model.predict(candidate)  # Negate because we maximize in Pyomo but SciPy minimizes

def scipy_optimization(x_row):
    """
    Optimization using SciPy.
    """
    # Extract problem data
    other_values = list(x_row[other_variables])  # Fixed values for other variables
    continuous_count = len(continuous_variables)
    binary_count = len(binary_variables)
    
    # Define bounds for variables
    bounds = [(0, 10), (0,100), (0,10), (0,5)] + [(0, 1)] * binary_count  # Continuous [0, 120], Binary [0, 1]
    
    # Define initial guesses
    x0 = list(x_row[continuous_variables + binary_variables])
    # x0 = [0] * len(continuous_variables + binary_variables)
    # x0 = list(x_row[continuous_variables]) + [0.5] * binary_count

    # Define constraints
    constraints = [
        {'type': 'ineq', 'fun': lambda vars: 10 - vars[continuous_count + 0]},  # y[0] <= 10
        {'type': 'ineq', 'fun': lambda vars: 100 - vars[continuous_count + 1]},  # y[1] <= 100
        {'type': 'ineq', 'fun': lambda vars: 10 - vars[continuous_count + 2]},  # y[2] <= 10
        {'type': 'ineq', 'fun': lambda vars: 5 - vars[continuous_count + 3]},  # y[3] <= 5
        {'type': 'ineq', 'fun': lambda vars: 1 - vars[continuous_count + 6] - vars[continuous_count + 7]},  # y[3] <= 5
    ]
    constraints = constraints + [{'type' : 'ineq', 'fun' : lambda vars: 1 - vars[continuous_count + i]} for i in range(binary_count)]

    # Solve using SciPy
    result = minimize(
        objective_function_scipy,
        x0=x0,
        args=(other_values, loaded_model, continuous_count, binary_count),
        bounds=bounds,
        constraints=constraints,
        # method='SLSQP',  # Sequential Least Squares Programming
        method = "COBYLA",
        options={'disp': True}
    )

    # Extract results
    optimized_vars = result.x
    continuous_vars = np.round(optimized_vars[:continuous_count])
    binary_vars = np.round(optimized_vars[continuous_count:])  # Round binary variables

    # Display results
    print("Optimization Result:")
    print("Continuous Variables:", continuous_vars)
    print("Binary Variables:", binary_vars)
    print("initial vars: ", list(x0))

    best_candidate = list(continuous_vars) + list(binary_vars) + other_values
    result_objective = loaded_model.predict(best_candidate) 
    print("Objective Value:", result_objective)

    return result_objective, list(continuous_vars) + list(binary_vars)

In [28]:
from concurrent.futures import ThreadPoolExecutor

def optimize_row(i):
    x_row = X_forecast.iloc[i]
    return scipy_optimization(x_row)

# Parallelize across all rows
with ThreadPoolExecutor() as executor:
    results = list(executor.map(optimize_row, range(X_forecast.shape[0])))

optimized_effienciencies, optimized_x = zip(*results)


Optimization Result:
Continuous Variables: [ 2. 20.  6.  4.]
Binary Variables: [1. 1. 1. 1. 1. 0. 1. 0.]
initial vars:  [2.0, 20.0, 5.0, 3.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.16015125097685431
Optimization Result:
Continuous Variables: [ 3. 30.  6.  5.]
Binary Variables: [0. 1. 1. 0. 1. 0. 1. 0.]
initial vars:  [2.0, 30.0, 5.0, 4.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.1826926073524482
Optimization Result:
Continuous Variables: [ 2. 30.  5.  5.]
Binary Variables: [0. 1. 1. 1. 1. 1. 1. 0.]
initial vars:  [2.0, 30.0, 5.0, 4.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
Objective Value: 0.18541252825096666
Optimization Result:
Continuous Variables: [ 2. 30.  2.  2.]
Binary Variables: [1. 1. 0. 1. 1. 0. 1. 0.]
initial vars:  [2.0, 30.0, 2.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.16806292470002976
Optimization Result:
Continuous Variables: [ 2. 15.  6.  3.]
Binary Variables: [0. 1. 1. 1. 1. 1. 1. 0.]
initial vars:  [2.0,

In [29]:
optimized_effienciencies = []
optimized_x = []
for i in range(X_forecast.shape[0]):
# for i in range(10):
    x_row = X_forecast.iloc[i]
    print(X_forecast.shape[0] - i)
    best_eff, best_x = scipy_optimization(x_row)
    optimized_effienciencies.append(best_eff)
    optimized_x.append(best_x)

4702
Optimization Result:
Continuous Variables: [ 2. 20.  6.  4.]
Binary Variables: [1. 1. 1. 1. 1. 0. 1. 0.]
initial vars:  [2.0, 20.0, 5.0, 3.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.16015125097685431
4701
Optimization Result:
Continuous Variables: [ 3. 30.  6.  5.]
Binary Variables: [0. 1. 1. 0. 1. 0. 1. 0.]
initial vars:  [2.0, 30.0, 5.0, 4.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.1826926073524482
4700
Optimization Result:
Continuous Variables: [ 2. 30.  5.  5.]
Binary Variables: [0. 1. 1. 1. 1. 1. 1. 0.]
initial vars:  [2.0, 30.0, 5.0, 4.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
Objective Value: 0.18541252825096666
4699
Optimization Result:
Continuous Variables: [ 2. 30.  2.  2.]
Binary Variables: [1. 1. 0. 1. 1. 0. 1. 0.]
initial vars:  [2.0, 30.0, 2.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.16806292470002976
4698
Optimization Result:
Continuous Variables: [ 2. 15.  6.  3.]
Binary Variables: [0. 1. 1. 1. 1. 1. 1.

In [30]:
y_pred = loaded_model.predict(X_forecast)
y_pred

array([0.15964715, 0.10291529, 0.12315811, ..., 0.12621358, 0.163995  ,
       0.11380516])

In [31]:
final = pd.DataFrame(optimized_x, columns = loaded_model.feature_names_[:len(continuous_variables) + len(binary_variables)])
final_1 = pd.concat([final, forecast[other_variables].reset_index(drop = True)], axis = 1)
final_1['event_id'] = forecast['event_id']
final_1['Optimized_efficiency'] = optimized_effienciencies
final_1['Initial_efficiency'] = y_pred
final_1['Uplift'] = final_1['Optimized_efficiency'] - final_1['Initial_efficiency']
final_1['Uplift_dollars'] = final_1['Uplift'] * final_1['bestbid_usd'] * final_1['item_name']

In [32]:
print(f"Uplift in effiency : {sum(final_1['Uplift_dollars']/sum(final_1['bestbid_usd'] * final_1['item_name']))*100} %")

Uplift in effiency : 5.637335404129601 %


In [33]:
final_1.to_csv(f"Optimized_result_Arun_Model_scipy_{zone}.csv", index = False)